In [1]:
# Colab: Install Required Libraries
!pip install  sentencepiece torch
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tdho4r79
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tdho4r79
  Resolved https://github.com/huggingface/transformers.git to commit 7bbc62474391aff64f63fcc064c975752d1fa4de
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Import Necessary Libraries
import torch
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# Load Prompt from a File
file_path = "call_114.txt"
file_name = file_path.split("/")[-1].split(".")[0]
with open(file_path, 'r') as file:
    prompt = file.read()

In [5]:
from transformers import pipeline
import torch

model_id = "meta-llama/Llama-3.2-3B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [6]:
messages = [
    {"role": "system", "content": "You are an specialist at text diarization of 911 emergency audio call"},
    {"role": "user", "content": prompt },
]

In [7]:
terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipe(
    messages,
    max_new_tokens=10000,
    eos_token_id=terminators,
    top_p=0.9,
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[0.00,6.30], Caller, Call 911.
[6.30,8.30], Caller, Hello?
[8.30,10.30], Caller, Hello 911?
[10.30,12.30], Caller, I need help.
[12.30,16.30], Caller, What happened? Can you get him to call?
[16.30,18.30], Caller, Call 911.
[18.30,20.30], Caller, Does he need a...
[20.30,22.30], Caller, He needs to run away.
[24.30,27.30], Caller, Does he need an ambulance or police?
[27.30,29.30], Caller, Okay.
[29.30,31.30], Caller, Ambulance or police?
[31.30,33.30], Caller, Ma'am?
[33.30,35.30], Caller, Hello?
[43.30,59.30], Caller, Friday, January 28, 2011. The time, 1014 AM.
[59.30,61.30], Caller, Hello?
[61.30,63.30], Caller, Hi, can I talk to your dad?
[63.30,65.30], Caller, Daddy, help!
[65.30,67.30], Caller, Okay, does he need an ambulance or police?
[67.30,69.30], Caller, Okay.
[69.30,71.30], Caller, No, ask him. Does he need police?
[71.30,75.30], Caller, If anybody gets right to you, you have to come right away. Okay?
[75.30,78.30], Caller, Ask him if he needs police or ambulance.
[78.30,8

In [8]:
import csv

rows = [line.replace('"', '').strip("[]").split(",",3) for line in assistant_response.strip().splitlines() if line]

# Specify the filename for the CSV file
filename = f"{file_name}_{model_id.replace('/', '_')}.csv"

# Write to CSV
with open(filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(rows)

print(f"CSV file '{filename}' created successfully.")

CSV file 'call_114_meta-llama_Llama-3.2-3B-Instruct.csv' created successfully.
